In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import spacy


PATH='/home/wk/myProjects/data/Enron/oper/'

TRN_PATH = 'train/'
VAL_PATH = 'test/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

bs=32; bptt=500
em_sz = 300  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
TEXT = pickle.load(open(f'{PATH}models/TEXT.pkl','rb'))

md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

opt_fn = partial(optim.Adam, betas=(0.7, 0.99))
learner = md.get_model(opt_fn, em_sz, nh, nl,
               dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

learner.load_encoder('a_adam3_10_enc')

In [2]:
### Test
ss=r"""

FYI -
 
A new (shell) Master Financial Agreement has been established in Global Contracts for EOL purposes
until the executed originals are received from legal.  I have updated this contract number in EOL's Profile Manager.  
Please call me at 5-4435 if you have any questions.  
 
Counterparty:
Public Utility District No. 1 of Benton County, Washington
CP ID Number:
51293
Contract Number:
96085521
Profile Manager Updated On:
10/26/01
 
 
Thanks,
Georgi
 
 
Georgi Landau
Enron Net Works
(Ph) 713/345-4435
(Fx)  713/646-2495
 
"""
s = [TEXT.preprocess(ss)] 
t=TEXT.numericalize(s)
' '.join(s[0])

ss = copy.deepcopy(s)
t=TEXT.numericalize(s)

m=learner.model

m[0].bs=1
m.eval()
m.reset()

res,*_ = m(t)
for i in range(len(s[0])):
    n=res[-1].topk(2)[1]    
    predict = TEXT.vocab.itos[n.data[0]]                      
    if (predict[0:2] == '@@'):
        print("pos=" + str(i) + "; " + predict + " <- " + s[0][i] , end=' \n')                    
        ss[0][i]=predict
        n.data[0] = TEXT.vocab.stoi[predict]
    else:
        n.data[0] = TEXT.vocab.stoi[s[0][i]]
    res,*_ = m(n[0].unsqueeze(0))
print('...')
print(len(t))

pos=0; @@othr_ph@@ <- fyi 
pos=39; @@othr_ph@@ <- 5-4435 
pos=58; @@othr_ph@@ <- 51293 
pos=61; @@othr_ph@@ <- 96085521 
pos=75; @@othr_ph@@ <- 713/345-4435 
pos=77; @@othr_ph@@ <- 713/646-2495 
...
78


In [3]:
### Test
ss=r"""
Attached is PIRA's latest "API Weekly Comment."

If you have any questions regarding the report's content, please contact
Dr. Gary Ross at (212) 686-6808, email: garyross@pira.com.

Contact Client Services regarding PIRA report distribution and address
changes at (212) 686-6808, email: support@pira.com.

NOTE: Circulation of the "API Weekly Comment" outside a Client's
licensed distribution area is strictly prohibited. Clients that are
unsure of their licensed distribution or require an extension of their
current license should contact their PIRA sales representative, or email
to sales@pira.com.

PIRA Energy Group
"""
s = [TEXT.preprocess(ss)] 
t=TEXT.numericalize(s)
' '.join(s[0])

ss = copy.deepcopy(s)
t=TEXT.numericalize(s)

m=learner.model

m[0].bs=1
m.eval()
m.reset()

res,*_ = m(t)
for i in range(len(s[0])):
    n=res[-1].topk(2)[1]    
    predict = TEXT.vocab.itos[n.data[0]]                      
    if (predict[0:2] == '@@'):
        print("pos=" + str(i) + "; " + predict + " <- " + s[0][i] , end=' \n')                    
        ss[0][i]=predict
        n.data[0] = TEXT.vocab.stoi[predict]
    else:
        n.data[0] = TEXT.vocab.stoi[s[0][i]]
    res,*_ = m(n[0].unsqueeze(0))
print('...')
print(len(t))

pos=22; @@othr_ph@@ <- (212) 
pos=25; @@othr_em@@ <- garyross@pira.com. 
pos=37; @@othr_ph@@ <- (212) 
pos=40; @@othr_em@@ <- support@pira.com. 
pos=82; @@othr_em@@ <- sales@pira.com. 
...
86


In [4]:
### Test
ss=r"""
Please use the next week to clean up our database of delivery locations and regions currently still in Enpower.   I would like to start the database clean with only the key hubs listed:

NEPOOL PTF
NY Zone J
NY Zone G
NY Zone A
Ontario
PJM East Hub
PJM West Hub
PJM West
Cinergy
Com-Ed
TVA
Entergy
SOCO
ERCOT South (FP)
ERCOT North (Basis)

Let's use this opportunity to clean up Enpower and start with a truly clean set of books.  
Team - Am I missing anthing?

Kevin Presto
Vice President, East Power Trading
Phone:  713-853-5035
Cell:  713-854-3923
Fax:  713-646-8272
"""
s = [TEXT.preprocess(ss)] 
t=TEXT.numericalize(s)
' '.join(s[0])

ss = copy.deepcopy(s)
t=TEXT.numericalize(s)

m=learner.model

m[0].bs=1
m.eval()
m.reset()

res,*_ = m(t)
for i in range(len(s[0])):
    n=res[-1].topk(2)[1]    
    predict = TEXT.vocab.itos[n.data[0]]                      
    if (predict[0:2] == '@@'):
        print("pos=" + str(i) + "; " + predict + " <- " + s[0][i] , end=' \n')                    
        ss[0][i]=predict
        n.data[0] = TEXT.vocab.stoi[predict]
    else:
        n.data[0] = TEXT.vocab.stoi[s[0][i]]
    res,*_ = m(n[0].unsqueeze(0))
print('...')
print(len(t))

pos=0; @@othr_ph@@ <- please 
pos=95; @@othr_ph@@ <- 713-853-5035 
pos=97; @@othr_ph@@ <- 713-854-3923 
pos=99; @@othr_ph@@ <- 713-646-8272 
...
100
